# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
csv_path=('../WeatherPy/output_data/cities.csv')

weather_data=pd.read_csv(csv_path)

weather_data

,City,Country,Date,Cloudiness,Humidity,Lat,Lon,Max Temp (°F),Wind Speed (mph)
0,Evensk,RU,2020-03-31 16:28:21,45,90,61.95,159.23,3.812,2.97521
1,Bambous virieux,MU,2020-03-31 16:24:33,75,88,-20.34,57.76,80.600,13.15356
2,Awjilah,LY,2020-03-31 16:29:41,0,31,29.11,21.29,72.392,18.25392
3,Hirara,JP,2020-03-31 16:25:48,1,89,24.80,125.28,74.822,5.68198
4,Bengkulu,ID,2020-03-31 16:27:59,100,79,-3.80,102.27,78.332,1.94619
...,...,...,...,...,...,...,...,...,...
564,Chulym,RU,2020-03-31 16:34:10,49,94,55.10,80.96,23.936,6.44256
565,Ayr,GB,2020-03-31 16:34:11,40,70,55.46,-4.63,51.008,13.86940
566,Verkhnevilyuysk,RU,2020-03-31 16:34:11,99,96,63.45,120.32,17.006,7.00181
567,Angoche,MZ,2020-03-31 16:34:12,5,79,-16.23,39.91,76.550,9.41777


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)
gmaps.figure()
figure_layout = {
    'width': '900px',
    'height': '900px'
}
coordinates = weather_data[['Lat','Lon']]
humidity = weather_data['Humidity'].astype(float)
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, dissipating=False, max_intensity=weather_data['Humidity'].max(), point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='900px', width='900px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
narrowed_df = weather_data.loc[(weather_data["Max Temp (°F)"]>70) & (weather_data["Max Temp (°F)"]<80) & (weather_data["Wind Speed (mph)"]<10) & (weather_data["Cloudiness"]==0)]
narrowed_city_df = narrowed_df.reset_index(drop=True)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
narrowed_city_df['Hotel Name'] = ''
narrowed_city_df.head(20)

,City,Country,Date,Cloudiness,Humidity,Lat,Lon,Max Temp (°F),Wind Speed (mph),Hotel Name
0,Idappadi,IN,2020-03-31 16:30:06,0,41,11.58,77.85,79.286,2.10278,
1,Dingle,PH,2020-03-31 16:27:31,0,75,11.00,122.67,78.998,6.84522,
2,Teknaf,BD,2020-03-31 16:30:41,0,68,20.86,92.31,74.300,5.43591,
3,Maryborough,AU,2020-03-31 16:31:00,0,88,-25.53,152.70,71.006,3.93712,
4,Mariental,NaN,2020-03-31 16:31:25,0,21,-24.63,17.97,79.772,4.89903,
5,Sur,OM,2020-03-31 16:31:34,0,68,22.57,59.53,74.984,1.36457,
6,Riyadh,SA,2020-03-31 16:27:29,0,24,24.69,46.72,78.800,4.69770,
7,Ahvaz,IR,2020-03-31 16:33:40,0,33,31.32,48.67,75.200,4.69770,
8,Hervey bay,AU,2020-03-31 16:34:09,0,94,-25.30,152.85,71.006,6.93470,


In [8]:
base_url= 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
hotel_names = []
params = {
    "radius":5000,
    "types": "hotel",
    "keyword": "hotel",
    "key":g_key}

for index, row in narrowed_city_df.iterrows():
    lat = row["Lat"]
    lng = row["Lon"]
    params["location"] = {f'{lat},{lng}'}
    hotels = requests.get(base_url, params=params).json()
    try:
        narrowed_city_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (KeyError, IndexError):
        narrowed_city_df.loc[index, "Hotel Name"] = np.nan


In [9]:
narrowed_city_df.dropna(inplace=True)
narrowed_city_df

,City,Country,Date,Cloudiness,Humidity,Lat,Lon,Max Temp (°F),Wind Speed (mph),Hotel Name
2,Teknaf,BD,2020-03-31 16:30:41,0,68,20.86,92.31,74.300,5.43591,Alo Resort
3,Maryborough,AU,2020-03-31 16:31:00,0,88,-25.53,152.70,71.006,3.93712,The Maryborough Motel and Conference Centre
5,Sur,OM,2020-03-31 16:31:34,0,68,22.57,59.53,74.984,1.36457,Sur Hotel
6,Riyadh,SA,2020-03-31 16:27:29,0,24,24.69,46.72,78.800,4.69770,فندق ميون ١٠٥
7,Ahvaz,IR,2020-03-31 16:33:40,0,33,31.32,48.67,75.200,4.69770,Ahwaz Pars Hotel
8,Hervey bay,AU,2020-03-31 16:34:09,0,94,-25.30,152.85,71.006,6.93470,Pier One Resort


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lon"]]

In [11]:
# Add marker layer ontop of heat map
hotel_markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(hotel_markers)
fig
# Display Map

Figure(layout=FigureLayout(height='900px', width='900px'))